# Facility Sizing Problem

Consider the Facility Sizing Problem described in Chapter 4 of the Benders Decomposition lecture notes.
The data of an instance of the problem is provided below, together with a class that represents the problem and a class that represents the full (non-decomposed) model.
- Q1. Solve the instance of the problem using Benders decomposition
- Q2. Ensure that the solution obtained is the same as the one give by the non-decomposed model.

## Data of an instance

In [1]:
import random as r
n_locations = 5
n_customers = 5

r.seed(1)
# Random capacity costs between 100 and 300
fixed_costs = [(100 + (r.random() * 200)) for i in range(n_locations)]

# Random delivery costs between 10 and 40
delivery_costs = {(i,j):(10 + r.random() * 30) for j in range(n_customers) for i in range(n_locations)}

# Random demands between 50 and 100
demands = [(50 + (r.random() * 50)) for j in range(n_customers)]

# Random capacities between 100 and 140
capacities = [(120 + (r.random() * 20)) for i in range(n_locations)]


## Class for the Facility Sizing Problem

In [2]:
class FacilitySizingProblem:

    def __init__(self,n_facilities:int,n_customers:int,fixed_costs:list,delivery_costs:dict,demands:list,capacity:list):
        self.n_facilities = n_facilities
        self.n_customers = n_customers
        self.fixed_costs = fixed_costs
        self.delivery_costs = delivery_costs
        self.demands = demands
        self.capacity = capacity
        


In [3]:
p = FacilitySizingProblem(n_locations, n_customers, fixed_costs, delivery_costs, demands, capacities)

## Class for the full model

In [4]:
from gurobipy import Model, GRB, quicksum

In [5]:
class FullModel:

    def __init__(self, fsp:FacilitySizingProblem):
        self.fsp = fsp
        self.m = Model()

        # Creates the variables
        self.y = self.m.addVars(fsp.n_facilities, fsp.n_customers, name="y")
        self.x = self.m.addVars(fsp.n_facilities, name="x")

        term1 = self.x.prod(fsp.fixed_costs)
        term2 = self.y.prod(fsp.delivery_costs)
        self.m.setObjective(term1+term2, GRB.MINIMIZE)

        # Constraints

        self.m.addConstrs(self.y.sum(i, '*') <= self.x[i] for i in range(fsp.n_facilities))
        self.m.addConstrs(self.y.sum('*', j) >= fsp.demands[j] for j in range(fsp.n_customers))
        self.m.addConstrs(self.x[i] <= self.fsp.capacity[i] for i in range(fsp.n_facilities))
    

    def solve(self):
        self.m.optimize()

    def print_solution(self):
        for i in range(self.fsp.n_facilities):
            print('%s %g' % (self.x[i].varName, self.x[i].x))
        print('Obj: %g' % self.m.objVal)
